In [ ]:
from datetime import date
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.random import gamma

df = pd.read_csv("数据源/order_train1.csv", encoding="gbk")
df["order_date"]=df["order_date"].apply(pd.to_datetime,format='%Y-%m-%d')
# data.drop('order_date', axis=1, inplace=True)

df = df.set_index('order_date')
df_train = df[['sales_region_code', 'item_code', 'first_cate_code','second_cate_code','ord_qty']]

In [ ]:
pre = pd.read_csv("predict_sku1.csv", encoding="gbk")
## 查看预测数据
data_pre = pd.DataFrame(pre,columns=['sales_region_code','item_code', 'first_cate_code', 'second_cate_code'])
# groupby(['sales_region_code'])['ord_qty'].sum().sort_values()

In [ ]:
data_pre.shape

In [ ]:
predictions_df = pd.DataFrame(columns=['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code', 'mse', 'prediction_1_month', 'prediction_2_month', 'prediction_3_month'])

In [ ]:
df_train1 = df_train[['sales_region_code', 'item_code', 'first_cate_code','second_cate_code']]

In [ ]:
df_train1 = df_train1.reset_index(drop=True)

In [ ]:
df_train1

预测集中4个特征的值在训练集中都有的数据，保存到pred-有.csv

In [ ]:
predictions_df = pd.DataFrame(columns=['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code', ])
for i in range(len(data_pre)):
    sales_region_code, item_code, first_cate_code, second_cate_code = data_pre.iloc[i, :]
    # print(sales_region_code, item_code, first_cate_code, second_cate_code)
    if len(df_train[(df_train['sales_region_code'] == sales_region_code) & (df_train['item_code'] == item_code) & (df_train['first_cate_code'] == first_cate_code) & (df_train['second_cate_code'] == second_cate_code)])>0:

        predictions_df.loc[i] = {'sales_region_code': sales_region_code,
                        'item_code': item_code,
                        'first_cate_code': first_cate_code,
                        'second_cate_code': second_cate_code}

predictions_df.to_csv("pred-有.csv", index=False, encoding='utf-8')
print(predictions_df.shape)

预测集中4个特征的值在训练集中不完全有的数据，保存到pred-无.csv

In [ ]:
predictions_df = pd.DataFrame(columns=['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code', ])
for i in range(len(data_pre)):
    sales_region_code, item_code, first_cate_code, second_cate_code = data_pre.iloc[i, :]
    # print(sales_region_code, item_code, first_cate_code, second_cate_code)
    if len(df_train[(df_train['sales_region_code'] == sales_region_code) & (df_train['item_code'] == item_code) & (df_train['first_cate_code'] == first_cate_code) & (df_train['second_cate_code'] == second_cate_code)])==0:

        predictions_df.loc[i] = {'sales_region_code': sales_region_code,
                        'item_code': item_code,
                        'first_cate_code': first_cate_code,
                        'second_cate_code': second_cate_code}

predictions_df.to_csv("pred-无.csv", index=False, encoding='utf-8')
print(predictions_df.shape)

预测集中item_code特征的值在训练集中不存在的数据，保存到pred-item_code.csv

In [ ]:
data_pre = pd.read_csv('pred-无.csv', encoding="gbk")

predictions_df = pd.DataFrame(columns=['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code', ])
for i in range(len(data_pre)):
    sales_region_code, item_code, first_cate_code, second_cate_code = data_pre.iloc[i, :]
    # print(sales_region_code, item_code, first_cate_code, second_cate_code)
    if item_code not in df_train['item_code'].unique():
        a = df_train[(df_train['sales_region_code'] == sales_region_code) & (df_train['first_cate_code'] == first_cate_code) & (df_train['second_cate_code'] == second_cate_code)]
        predictions_df.loc[i] = {'sales_region_code': sales_region_code,
                        'item_code': item_code,
                        'first_cate_code': first_cate_code,
                        'second_cate_code': second_cate_code}

predictions_df.to_csv("pred-item_code.csv", index=False, encoding='utf-8')
print(predictions_df.shape)

pred-无和pred-item_code中不同的值，保存到different_rows.csv

In [ ]:
import pandas as pd
 # 读取 pred 和 pred-item_code 数据框
pred_df = pd.read_csv('pred-无.csv')
item_code_df = pd.read_csv('pred-item_code.csv')
 # 合并两个数据框，使用 left join
merged_df = pred_df.merge(item_code_df, on='item_code', how='left', indicator=True)
 # 筛选出只在左侧数据框出现的行（即 pred-无 中有、而 pred-item_code 中没有的行）
different_rows = merged_df[merged_df['_merge'] == 'left_only'].drop_duplicates()
different_rows.shape
 # 保存为新的表
different_rows.to_csv('different_rows.csv', index=False)

保存只有后3个特征在训练集中有，sales_region_code没有的代码

In [12]:
import pandas as pd

df_diff = pd.read_csv('different_rows.csv', usecols=['sales_region_code', 'item_code', 'first_cate_code', 'second_cate_code'])

df_merge = pd.merge(df_diff, df_train, on=['item_code', 'first_cate_code', 'second_cate_code'], how='left')

df_result = df_merge[df_merge['sales_region_code_x'] != df_merge['sales_region_code_y']]
df_result.drop(['sales_region_code_y','ord_qty'], axis=1, inplace=True)
df_result.drop_duplicates(subset=['sales_region_code_x', 'item_code', 'first_cate_code', 'second_cate_code'],keep='first', inplace=True)
df_result.rename(columns={'sales_region_code_x': 'sales_region_code'})
df_result.to_csv('pred-sales_region_code.csv', index=False)
print(df_result.shape)

(104, 4)
